In [5]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/d3/9d/df59bd657de871cd3429d58ccdf1479f13e9aa18c10b5051b2c14249ebbc/tensorflow-2.13.0-cp310-cp310-macosx_10_15_x86_64.whl.metadata
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.1.21 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Obtaining dependency information for h5py>=2.9.0 from https://files.pythonhosted.org/packages/df/fe/3809103d284595bbc07c1568b4dd10f4954049c7b3d5c922d9dd15256994/h5py-3.9.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata
  Using cached h5py-3.9.0-cp310

  Using cached pyasn1-0.5.0-py2.py3-none-any.whl (83 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
Using cached tensorflow-2.13.0-cp310-cp310-macosx_10_15_x86_64.whl (216.2 MB)
Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
Using cached h5py-3.9.0-cp310-cp310-macosx_10_9_x86_64.whl (3.2 MB)
Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
Using cached libclang-16.0.6-py2.py3-none-macosx_10_9_x86_64.whl (24.5 MB)
Using cached protobuf-4.23.4-cp37-abi3-macosx_10_9_universal2.whl (400 kB)
Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
Using cached google_auth-2.22.0-py2.py3-none-any.whl (181 kB)
Using cached Markdown-3.4.4-py3-none-any.whl (94 kB)
Using cached tensorboard_data_server-0.7.1-py3-none-macosx_10_9_x86_64.whl (4.8 MB)
Using cached Werkzeug-2.3.6-py3-none-any.whl (242 kB)
Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
anceled
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use update

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from losses import DICELossMultiClass

from torch.autograd import Variable
from torch.utils.data import DataLoader

from torch.utils.data.dataset import Dataset
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import os
import sys


#from data import DatasetLSTM
from CLSTM import CLSTM
from data import DatasetLSTM


from stardist.models import StarDist2D
from lamprogen.recipes.dl import stardist_2d_slicewise

RuntimeError: Please install TensorFlow: https://www.tensorflow.org/install/

## Define Dataset and Model

In [ ]:
## import stardist model in place of ku-net
modeldir = f'/Users/marcchiu/lamprogen-python/notebooks/deep learning/models'
name = 'lamprogen-stardist-trained'
model_stardist = StarDist2D(None, name=name,basedir=modeldir)

# load BDCLSTM Model
model = CLSTM(input_channels=1, hidden_channels=[64])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
criterion = DICELossMultiClass()
model = model.to(device)
criterion = criterion.to(device)


In [ ]:
path = 'Fixed'
train_data = DatasetLSTM(path)
test_data = DatasetLSTM(path)

In [ ]:
train_data[0][0] = stardist_2d_slicewise(train_data[0][0], model_stardist)

In [ ]:
batch_size = 1
train_iterator = torch.utils.data.DataLoader(train_data,batch_size=batch_size ,shuffle=True)
valid_iterator = torch.utils.data.DataLoader(test_data,batch_size=batch_size ,shuffle=False)


## Define Traing Loop

In [ ]:
# # Define Training Loop

def train(epoch, iterator, optimizer, criterion):
    model.train()
    for image, mask in iterator:
            
        image = image.to(device)
        mask = mask.to(device)

        optimizer.zero_grad()

        output = model(labels)
        loss = criterion(output, mask)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        #epoch_acc += acc.item()

    return epoch_loss / len(iterator), #epoch_acc / len(iterator)

In [ ]:
train(5, train_iterator, optimizer, criterion)

## Define Testing Loop

In [ ]:
def evaluate(epoch, eval_iterator, optimizer, criterion):
    model.eval()
    min_loss=sys.maxsize
    for image, mask in eval_iterator:

        image = image.to(device)
        mask = mask.to(device)

        optimizer.zero_grad()

        labels = stardist_2d_slicewise(image, model_startdist)
        output = model(labels)
        loss = criterion(output, mask)

        loss.backward()
        optimizer.step()
    
        if loss<min_loss :
            torch.save(model.state_dict(), "models")